# OpenAI Agents SDK

### **Оглавление**

1. Что такое агент?
2. Что такое контекст агента и зачем он нужен?
3. Что такое инструменты (tools) в многоагентной системе?
4. Что такое `RunContextWrapper`
5. Что такое `Agent`
6. Что такое `@input_guardrail` и зачем он нужен?
7. Что такое `handoff`

### **Введение в OpenAI Agents SDK**

**OpenAI Agents SDK** — это новый фреймворк для создания интеллектуальных агентов, который превращает сложные мультиагентные системы в управляемые и масштабируемые решения. Если вы когда-либо сталкивались с проблемами цикличности агентов, ручной оркестрации инструментов или хрупких валидаций — этот SDK создан для вас. В этом туториале мы разберем его архитектуру, ключевые компоненты и преимущества перед аналогами вроде LangChain.  

### 🔍 Почему Agents SDK появился сейчас?  
1. **Эволюция ИИ-систем**: переход от чат-ботов к автономным агентам, способным планировать и действовать, требует новых инструментов. Традиционные подходы (например, LangChain) страдают от:  
   - Цикличности агентов из-за нечетких условий завершения.  
   - Сложности интеграции валидаций и передачи задач между агентами.    
   SDK решает это через **минималистичные абстракции** и **готовые паттерны оркестрации** .  

2. **Контекст разработки**:  
   - OpenAI постепенно заменяет Assistants API на **Responses API** — более гибкую систему для агентов с поддержкой встроенных инструментов (веб-поиск, RAG, работа с ПК) .  
   - Agents SDK стал «прослойкой» между этим API и разработчиком, упрощая создание сложных потоков .  

### ⚙️ Ключевые компоненты: что под капотом?  
В основе SDK лежат четыре принципа, которые делают агентов **надежными** и **расширяемыми**:  


### 💡 Почему это прорыв?  
- **Упрощение сложного**: раньше мультиагентные системы требовали кастомной оркестрации (например, через LangGraph). Теперь handoffs и guardrails инкапсулируют эту логику .  
- **Производительность**: агенты используют **GPT-4o** с оптимизированными промптами, что снижает «галлюцинации» .  
- **Гибкость**: поддержка сторонних LLM (Gemini, Claude через OpenAI-совместимые API) .  
- **Готовые инструменты**: веб-поиск, File Search, RAG — из коробки .  

### 🚀 Примеры реального использования  
- **Голосовые ассистенты** (например, обработка запросов через микрофон → агент-роутинг → ответ в TTS).  
- **Durable Agents**: агенты с постоянным состоянием, работающие на инфраструктуре Cloudflare.  
- **Автоматизация рабочих процессов**: например, обработка заявок с проверками через guardrails.  

### 🔮 Что дальше в туториале?  
В первую очередь мы с тобой разберёмся, как работает OpenAI Agents SDK под капотом 👇 

```python
# Библиотеки OpenAI Agents SDK
from agents import (
    Agent,
    GuardrailFunctionOutput,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    function_tool,
    handoff,
    input_guardrail,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
```

## 🤖 1. Что такое агент?

Хочу начать с того, что дадим определение тому, что такое вообще агент или мультиагентная система. Тут мы позаимствоваем определение из тутора от Сбера:



## 🧠 2. Что такое контекст агента и зачем он нужен?

В многоагентных системах (multi-agent systems), каждый агент — это как участник командной работы. Чтобы каждый агент мог эффективно взаимодействовать с другими, он должен понимать текущую ситуацию: кто клиент, какой у него номер рейса, где он сидит, подтверждено ли бронирование и т.д. Всё это называется контекстом.

Контекст — это не просто "данные пользователя". Это состояние, в котором находится агент во время сессии. Контекст можно сравнить с "рабочей памятью" агента — он постоянно обновляется по мере общения и передаётся между агентами, чтобы сохранить целостность обслуживания.

In [ ]:
# =========================
# КОНТЕКСТЫ И МОДЕЛИ ДАННЫХ
# =========================
class AirlineAgentContext(BaseModel):
    """
    Контекст для агентов клиентского сервиса авиакомпании.
    
    Description:
    ---------------
        Класс для хранения информации о клиенте и состоянии
        разговора между различными агентами системы.
    
    Args:
    ---------------
        passenger_name: Имя пассажира
        confirmation_number: Номер подтверждения бронирования
        seat_number: Номер места в самолете
        flight_number: Номер рейса
        account_number: Номер аккаунта клиента
    
    Returns:
    ---------------
        BaseModel: Экземпляр контекста агента
    """
    
    passenger_name:      Optional[str] = None
    confirmation_number: Optional[str] = None
    seat_number:         Optional[str] = None
    flight_number:       Optional[str] = None
    account_number:      Optional[str] = None


# Создание начального контекста для новой сессии
def create_initial_context() -> AirlineAgentContext:
    """
    Фабрика для создания нового контекста агента.
    
    Description:
    ---------------
        Создает новый экземпляр AirlineAgentContext с сгенерированным
        номером аккаунта.
    
    Returns:
    ---------------
        AirlineAgentContext: Новый экземпляр контекста с базовыми данными
    """
    ctx = AirlineAgentContext()
    # Генерируем случайный номер аккаунта для демо-версии
    ctx.account_number = str(random.randint(10000000, 99999999))
    return ctx

## 🛠️ 3. Что такое инструменты (tools) в многоагентной системе?

Инструменты — это обычные Python-функции, которые агенты вызывают **не напрямую**, а через фреймворк. Это позволяет LLM выполнять действия: менять место, проверять рейс, отвечать на вопросы и т.п. Такие функции должны быть оформлены специальным образом — через `@function_tool`.


### 🧠 Как работает `@function_tool` под капотом

`@function_tool` превращает вашу функцию (например, `update_seat`) в объект `FunctionTool`, который:

* понятен модели (имеет имя, описание и схему параметров);
* может быть вызван фреймворком на основе JSON-запроса от LLM;
* обеспечивает строгую проверку аргументов, обработку ошибок и совместимость с контекстом агента.


### ✅ Как именно это происходит

#### 1. 🔍 Анализ функции

Ты пишешь:

```python
@function_tool
async def update_seat(
    context: RunContextWrapper[AirlineAgentContext], 
    confirmation_number: str, 
    new_seat: str
) -> str:
    ...
```

SDK вызывает:

```python
schema = function_schema(func=update_seat)
```

Из docstring и аннотаций извлекается:

```json
{
  "name": "update_seat",
  "description": "Обновление места пассажира по номеру подтверждения.",
  "parameters": {
    "confirmation_number": { "type": "string" },
    "new_seat": { "type": "string" }
  }
}
```


#### 2. 🧩 Генерация обёртки вызова

SDK создаёт функцию:

```python
async def _on_invoke_tool(ctx: ToolContext, input: str):
    json_data = json.loads(input)
    parsed = schema.params_pydantic_model(**json_data)
    args, kwargs = schema.to_call_args(parsed)
    return await update_seat(ctx, *args, **kwargs)
```

Эта обёртка:

* принимает JSON от LLM (`{"confirmation_number": "ABC123", "new_seat": "16A"}`),
* валидирует его через `pydantic`,
* вызывает `update_seat(...)`,
* возвращает результат (строку) обратно модели.


#### 3. ⚠️ Обработка ошибок

Если в JSON ошибка (например, `"new_seat": 42`), фреймворк:

* возвращает LLM сообщение: `"Invalid input: new_seat must be a string"`
* или вызывает fallback-функцию, если задана.


#### 4. 🧱 Регистрация как `FunctionTool`

SDK создаёт объект:

```python
FunctionTool(
    name="update_seat",
    description="...",
    params_json_schema=...,
    on_invoke_tool=_on_invoke_tool
)
```

Этот объект добавляется в список доступных инструментов агента.

### 📌 Пример работы в рантайме

Допустим, пользователь спрашивает:

> "Могу я поменять своё место на 16A?"

LLM решает вызвать:

```json
{
  "name": "update_seat",
  "arguments": {
    "confirmation_number": "ABC123",
    "new_seat": "16A"
  }
}
```

SDK вызывает:

```python
await update_seat(context, confirmation_number="ABC123", new_seat="16A")
```

и возвращает:

```json
"Updated seat to 16A for confirmation number ABC123"
```

### 🎯 Другие примеры в твоём коде

| Функция              | Назначение                                        |
| -------------------- | ------------------------------------------------- |
| `faq_lookup_tool`    | Ответы на часто задаваемые вопросы (багаж, Wi-Fi) |
| `flight_status_tool` | Проверка статуса рейса по номеру                  |
| `baggage_tool`       | Информация о нормативах и сборах на багаж         |
| `display_seat_map`   | Запуск UI-карты мест                              |

Во всех случаях — та же логика:

1. Вызов от LLM в JSON
2. Обёртка фреймворка парсит и валидирует
3. Вызывается твоя функция
4. Возвращается результат модели

### 📦 Вывод

`@function_tool`:

- ✅ Делает функцию доступной LLM
- ✅ Добавляет описание и схему параметров
- ✅ Обеспечивает строгую валидацию
- ✅ Поддерживает контекст
- ✅ Оборачивает вызов через JSON-интерфейс

> **💡 Главная идея**: ты пишешь *бизнес-логику*, фреймворк берёт на себя всё остальное — трансформацию, проверку, интеграцию с агентом и взаимодействие с LLM.

In [ ]:
# =========================
# ИНСТРУМЕНТЫ (TOOLS)
# =========================
# Инструмент для поиска ответов на часто задаваемые вопросы
@function_tool(
    name_override="faq_lookup_tool", 
    description_override="Lookup frequently asked questions."
)
async def faq_lookup_tool(question: str) -> str:
    """
    Поиск ответов на часто задаваемые вопросы.
    
    Description:
    ---------------
        Функция для поиска предопределенных ответов на популярные
        вопросы клиентов о багаже, местах в самолете и Wi-Fi.
    
    Args:
    ---------------
        question: Вопрос клиента в виде строки
    
    Returns:
    ---------------
        str: Ответ на вопрос или сообщение о том, что ответ не найден
    """
    q = question.lower()
    
    # Проверяем вопросы о багаже
    if "bag" in q or "baggage" in q:
        return (
            "You are allowed to bring one bag on the plane. "
            "It must be under 50 pounds and 22 inches x 14 inches x 9 inches."
        )
    # Проверяем вопросы о местах в самолете
    elif "seats" in q or "plane" in q:
        return (
            "There are 120 seats on the plane. "
            "There are 22 business class seats and 98 economy seats. "
            "Exit rows are rows 4 and 16. "
            "Rows 5-8 are Economy Plus, with extra legroom."
        )
    # Проверяем вопросы о Wi-Fi
    elif "wifi" in q:
        return "We have free wifi on the plane, join Airline-Wifi"
    
    return "I'm sorry, I don't know the answer to that question."


# Инструмент для обновления места пассажира
@function_tool
async def update_seat(
    context: RunContextWrapper[AirlineAgentContext], 
    confirmation_number: str, 
    new_seat: str
) -> str:
    """
    Обновление места пассажира по номеру подтверждения.
    
    Description:
    ---------------
        Функция для изменения номера места пассажира в контексте
        разговора и возврата подтверждения операции.
    
    Args:
    ---------------
        context: Обертка контекста с информацией о пассажире
        confirmation_number: Номер подтверждения бронирования
        new_seat: Новый номер места
    
    Returns:
    ---------------
        str: Подтверждение успешного обновления места
    
    Raises:
    ---------------
        AssertionError: Если номер рейса не указан в контексте
    """
    # Обновляем информацию в контексте
    context.context.confirmation_number = confirmation_number
    context.context.seat_number = new_seat
    
    # Проверяем наличие номера рейса
    assert context.context.flight_number is not None, \
        "Flight number is required"
    
    return (f"Updated seat to {new_seat} for confirmation number "
            f"{confirmation_number}")


# Инструмент для проверки статуса рейса
@function_tool(
    name_override="flight_status_tool",
    description_override="Lookup status for a flight."
)
async def flight_status_tool(flight_number: str) -> str:
    """
    Получение статуса рейса по номеру.
    
    Description:
    ---------------
        Функция для получения актуальной информации о статусе
        рейса, времени вылета и номере гейта.
    
    Args:
    ---------------
        flight_number: Номер рейса для проверки
    
    Returns:
    ---------------
        str: Информация о статусе рейса
    """
    return (f"Flight {flight_number} is on time and scheduled to "
            f"depart at gate A10.")


# Инструмент для информации о багаже
@function_tool(
    name_override="baggage_tool",
    description_override="Lookup baggage allowance and fees."
)
async def baggage_tool(query: str) -> str:
    """
    Получение информации о багаже и тарифах.
    
    Description:
    ---------------
        Функция для получения информации о допустимом весе багажа,
        размерах и дополнительных сборах.
    
    Args:
    ---------------
        query: Запрос о багаже от клиента
    
    Returns:
    ---------------
        str: Информация о багаже или запрос на уточнение
    """
    q = query.lower()
    
    # Проверяем вопросы о сборах
    if "fee" in q:
        return "Overweight bag fee is $75."
    # Проверяем вопросы о нормах
    if "allowance" in q:
        return ("One carry-on and one checked bag (up to 50 lbs) "
                "are included.")
    
    return "Please provide details about your baggage inquiry."


# Инструмент для отображения карты мест
@function_tool(
    name_override="display_seat_map",
    description_override="Display an interactive seat map to the customer."
)
async def display_seat_map(
    context: RunContextWrapper[AirlineAgentContext]
) -> str:
    """
    Отображение интерактивной карты мест.
    
    Description:
    ---------------
        Функция для запуска отображения интерактивной карты мест
        в пользовательском интерфейсе, где клиент может выбрать
        предпочтительное место.
    
    Args:
    ---------------
        context: Обертка контекста с информацией о пассажире
    
    Returns:
    ---------------
        str: Команда для UI для отображения карты мест
    """
    # Возвращаемая строка интерпретируется UI для открытия карты мест
    return "DISPLAY_SEAT_MAP"

## 🧠 4. Что такое `RunContextWrapper`

`RunContextWrapper[TContext]` — это **обёртка вокруг пользовательского контекста**, которая:

* передаётся во все инструменты, хуки и колбэки;
* содержит пользовательские данные (например, `flight_number`, `confirmation_number`);
* предоставляет объект `usage` с данными об использовании модели (опционально);
* **никогда не передаётся LLM** — это часть backend-логики.

```python
@dataclass
class RunContextWrapper(Generic[TContext]):
    context: TContext
    usage: Usage = field(default_factory=Usage)
```


### 📦 Где используется `RunContextWrapper`

В твоем коде:

```python
@function_tool
async def display_seat_map(
    context: RunContextWrapper[AirlineAgentContext]
) -> str:
```

Ты получаешь объект `RunContextWrapper`, внутри которого:

```python
context.context  → AirlineAgentContext
context.usage    → (данные о токенах, опционально)
```

Так ты можешь:

* читать и писать значения в пользовательский контекст (`context.context.flight_number`)
* не передавать эти данные модели
* использовать единый интерфейс во всех инструментах и колбэках


### 🧩 Как это работает в рантайме

Когда агент запускается:

```python
agent.run(context=AirlineAgentContext(...))
```

SDK автоматически оборачивает `context` в `RunContextWrapper`:

```python
wrapped = RunContextWrapper(context=your_airline_context_instance)
```

Далее он передаётся:

* в инструмент:

  ```python
  async def update_seat(context: RunContextWrapper, ...)
  ```
* в хук:

  ```python
  async def on_seat_booking_handoff(context: RunContextWrapper)
  ```
* и даже в `is_enabled`, если ты динамически включаешь/отключаешь инструменты


### 🧪 Пример из твоего кода

```python
context.context.flight_number = f"FLT-{random.randint(100, 999)}"
context.context.confirmation_number = ...
```

Ты работаешь со структурой `AirlineAgentContext` как с обычным `pydantic`-объектом. Например:

```python
@dataclass
class AirlineAgentContext:
    flight_number: str
    confirmation_number: str
```

Все изменения, сделанные в `context.context`, сохраняются и доступны другим агентам, инструментам и хукам в течение всей сессии.


### 🔒 Почему не передаётся LLM

Из документации:

> *"Contexts are not passed to the LLM. They're a way to pass dependencies and data to code you implement..."*

Это критически важно для:

* защиты приватных данных (PII)
* изоляции бизнес-логики
* стабильной передачи состояния между агентами


### 🧠 Краткий итог

| Поле                | Назначение                                        |
| ------------------- | ------------------------------------------------- |
| `context.context`   | Объект пользовательского состояния                |
| `context.usage`     | Информация об использовании токенов (опционально) |
| Используется в      | tools, hooks, `is_enabled`, approvals и др.       |
| Доступен внутри     | всех функций, обёрнутых через `@function_tool`    |
| Не передаётся в LLM | ✅ (только локальное состояние)                    |


### ✅ Вывод

`RunContextWrapper` — это **обёртка поверх состояния агента (`AirlineAgentContext`)**, которая:


#### ✅ Выполняет 3 ключевые роли:

1. **📦 Единый интерфейс передачи состояния**

   Вместо того чтобы передавать «голый» `AirlineAgentContext`, SDK всегда передаёт `RunContextWrapper`, чтобы:

   * унифицировать вызовы инструментов, хуков, `is_enabled`;
   * дать доступ к `.context` (твои данные) и `.usage` (вспомогательная телеметрия).


2. **🧭 Изолирует состояние агента от LLM**

   * Всё, что в `.context`, **никогда не отправляется в LLM**.
   * Это «рабочая память» агента, доступная только инструментам и логике SDK.


3. **🧰 Добавляет служебную информацию SDK**

   Например:

   ```python
   context.usage.total_tokens
   ```

   Ты можешь отслеживать использование токенов прямо в функциях — это доступно только благодаря обёртке.


#### 🔁 Пример потока данных:

```python
initial = AirlineAgentContext(...)
wrapper = RunContextWrapper(context=initial)
```

Этот `wrapper`:

* передаётся в `@function_tool`:

  ```python
  async def update_seat(context: RunContextWrapper[AirlineAgentContext], ...)
  ```

* доступен агентам, хукам и инструментам

* сохраняет согласованность данных по всей цепочке вызовов


### 🧠 Итог:

> **`RunContextWrapper` — это обёртка-сервис вокруг состояния агента, которая обеспечивает корректную, безопасную и согласованную передачу контекста во всей экосистеме OpenAI Agents SDK.**


In [ ]:
# Инструмент для отображения карты мест
@function_tool(
    name_override="display_seat_map",
    description_override="Display an interactive seat map to the customer."
)
async def display_seat_map(
    context: RunContextWrapper[AirlineAgentContext]
) -> str:
    """
    Отображение интерактивной карты мест.
    
    Description:
    ---------------
        Функция для запуска отображения интерактивной карты мест
        в пользовательском интерфейсе, где клиент может выбрать
        предпочтительное место.
    
    Args:
    ---------------
        context: Обертка контекста с информацией о пассажире
    
    Returns:
    ---------------
        str: Команда для UI для отображения карты мест
    """
    # Возвращаемая строка интерпретируется UI для открытия карты мест
    return "DISPLAY_SEAT_MAP"


# =========================
# HOOKS (ПЕРЕХВАТЧИКИ СОБЫТИЙ)
# =========================
# Перехватчик при передаче управления агенту бронирования
async def on_seat_booking_handoff(
    context: RunContextWrapper[AirlineAgentContext]
) -> None:
    """
    Обработчик передачи управления агенту бронирования мест.
    
    Description:
    ---------------
        Устанавливает случайные номер рейса и подтверждения при
        передаче управления агенту бронирования мест для демо-версии.
    
    Args:
    ---------------
        context: Обертка контекста для модификации данных
    """
    # Генерируем случайный номер рейса для демо
    context.context.flight_number = f"FLT-{random.randint(100, 999)}"
    # Генерируем случайный номер подтверждения
    context.context.confirmation_number = "".join(
        random.choices(string.ascii_uppercase + string.digits, k=6)
    )

## 🧠 5. Что такое `Agent`

`Agent` — это **интерфейс между LLM и твоей логикой (контекст, инструменты, guardrails и handoffs)**. Он описывает, *что* должен делать LLM, *с какими инструментами*, *в каких рамках* и *в каком контексте*.

Пример из твоего кода:

```python
guardrail_agent = Agent(
    model="gpt-4.1-mini",
    name="Relevance Guardrail",
    instructions="Определи, является ли сообщение пользователя ...",
    output_type=RelevanceOutput,
)
```

Создание агента — это конфигурация:

> 💬 «Вот что ты должен делать, с кем взаимодействовать и как именно это делать»


### 🧱 Из чего состоит агент?

| Атрибут             | Назначение                                                                |
| ------------------- | ------------------------------------------------------------------------- |
| `name`              | Название агента, может использоваться в UI или логах                      |
| `model`             | Конкретная модель OpenAI (например, `"gpt-4.1-mini"`)                     |
| `instructions`      | System prompt, описывающий поведение агента                               |
| `output_type`       | Ожидаемый тип ответа (например, `Pydantic модель`)                        |
| `tools`             | Список `FunctionTool` объектов, доступных агенту                          |
| `handoffs`          | Другие агенты, которым можно делегировать диалог                          |
| `input_guardrails`  | Проверки перед выполнением промпта                                        |
| `output_guardrails` | Проверки после генерации ответа                                           |
| `hooks`             | Callback-и жизненного цикла                                               |
| `tool_use_behavior` | Стратегия использования инструментов                                      |
| `context`           | (не часть `Agent`, но передаётся при запуске — через `RunContextWrapper`) |


### 🔄 Что происходит при создании агента?

Когда ты вызываешь:

```python
guardrail_agent = Agent(
    model="gpt-4.1-mini",
    name="Relevance Guardrail",
    instructions="...",
    output_type=RelevanceOutput
)
```

👉 создаётся экземпляр:

```python
Agent(
    name="Relevance Guardrail",
    model="gpt-4.1-mini",
    instructions="...",
    output_type=RelevanceOutput
)
```

➡️ SDK просто сохраняет эту конфигурацию до момента запуска.


### 🚀 Когда агент "запускается"?

При вызове:

```python
await Runner.run(
    starting_agent=guardrail_agent,
    input=user_message,
    context=AirlineAgentContext(...)
)
```

🔁 происходит следующее:

1. **Создаётся `RunContextWrapper`** вокруг твоего контекста
2. Вызывается `agent.get_system_prompt(...)` — генерируется промпт
3. Вызывается `agent.get_all_tools(...)` — активируются доступные инструменты
4. SDK отправляет промпт и input → LLM
5. LLM:

   * либо возвращает `final_output`,
   * либо вызывает инструмент (`tool_call`)
6. Результат инструмента подставляется обратно в prompt
7. LLM продолжает работу или возвращает результат
8. Применяются `output_guardrails`, если заданы
9. Финальный результат сериализуется в `output_type`, если задан


### 📌 Особенности реализации

#### 1. Инструкции могут быть динамическими

```python
instructions = lambda ctx, agent: f"Рейс пользователя: {ctx.context.flight_number}"
```

SDK поддерживает как строку, так и функцию — синхронную или асинхронную. Это позволяет динамически менять system prompt.


#### 2. Агента можно превратить в инструмент

```python
tool = agent.as_tool(
    tool_name="relevance_guardrail",
    tool_description="Проверка релевантности пользовательского сообщения"
)
```

Создаётся `FunctionTool`, который внутри вызывает `Runner.run(starting_agent=agent, ...)`.


#### 3. Типизация вывода (`output_type`)

Если ты передал `output_type=RelevanceOutput`, SDK:

* парсит `LLM output → pydantic модель`
* выбрасывает ошибку, если валидация не проходит

Это безопасный способ строго задать формат ответа агента.


#### 🧠 Вывод

Агент — это **объект-конфигурация**, определяющий поведение LLM:

* Он **не вызывает модель напрямую**,
* Он **не содержит данных** — только правила поведения,
* Он **не знает про input или контекст**, пока не будет запущен через `Runner.run(...)`.


### 📦 Архитектурная диаграмма (упрощённо):

```
          ┌─────────────┐
          │  Agent      │   ←─ ты настраиваешь (model, tools, prompt, ...)
          └─────┬───────┘
                │
        Runner.run(...)  ←─ ты запускаешь
                │
        ┌───────▼───────────────┐
        │ RunContextWrapper     │   ← обёртка над AirlineAgentContext
        └───────┬───────────────┘
                │
        ┌───────▼───────────────┐
        │     LLM (gpt-4.1)     │
        └───────┬───────────────┘
                │ tool_call?
          ┌─────▼──────┐
          │ FunctionTool│   ← ты определяешь (например, update_seat)
          └────────────┘
```

In [ ]:
# Агент для бронирования и изменения мест
seat_booking_agent = Agent[AirlineAgentContext](
    name="Seat Booking Agent",
    model="gpt-4.1",
    handoff_description="Полезный агент, который может изменить место в рейсе.",
    instructions=seat_booking_instructions,
    tools=[update_seat, display_seat_map],
    input_guardrails=[relevance_guardrail, jailbreak_guardrail],
)

## 🛡️ 6. Что такое `@input_guardrail` и зачем он нужен?

Это **декоратор**, превращающий обычную Python-функцию в **"предохранитель" (guardrail)** для агента, который:

* проверяет входное сообщение пользователя **до запуска LLM**,
* может **остановить выполнение агента**, если вход не проходит проверку,
* возвращает информацию через специальную структуру `GuardrailFunctionOutput`.


### ✅ Что делает `@input_guardrail` под капотом

#### Ты пишешь:

```python
@input_guardrail(name="Relevance Guardrail")
async def relevance_guardrail(
    context: RunContextWrapper[None], 
    agent: Agent, 
    input: Union[str, List[TResponseInputItem]]
) -> GuardrailFunctionOutput:
    ...
```

➡️ SDK превращает эту функцию в экземпляр:

```python
InputGuardrail(guardrail_function=relevance_guardrail, name="Relevance Guardrail")
```

Теперь `relevance_guardrail` — не просто функция, а **объект**, который:

* вызывается автоматически до запуска агента,
* имеет `.run(...)` метод,
* управляет логикой остановки выполнения.


### 🔁 Как работает `InputGuardrail.run(...)`

```python
output = await guardrail.guardrail_function(context, agent, input)
```

Результат должен быть экземпляром:

```python
GuardrailFunctionOutput(
    output_info=...,              # произвольный объект, например RelevanceOutput
    tripwire_triggered=True/False # True → агент не будет вызван
)
```

Если `tripwire_triggered=True` → SDK **прерывает агента** и бросает исключение `InputGuardrailTripwireTriggered`.


### 🔬 Пример из твоего кода

```python
result = await Runner.run(guardrail_agent, input, context=context.context)
final = result.final_output_as(RelevanceOutput)

return GuardrailFunctionOutput(
    output_info=final,
    tripwire_triggered=not final.is_relevant
)
```

Если `is_relevant=False`, выполнение агента будет остановлено, и результатом всей цепочки станет `final`.


### 📦 Что такое `GuardrailFunctionOutput`

Это **унифицированный контракт ответа guardrail-функции**.

```python
@dataclass
class GuardrailFunctionOutput:
    output_info: Any                 # необязательная информация о проверке
    tripwire_triggered: bool         # сработал ли "детонатор"
```

* `output_info` можно использовать для логирования, UI или аудита.
* `tripwire_triggered=True` говорит SDK: **«Остановись!»**


### 📌 Где используется

| Guardrail           | Когда срабатывает                    | Пример применения                 |
| ------------------- | ------------------------------------ | --------------------------------- |
| `@input_guardrail`  | **Перед** запуском агента            | Фильтрация спама/нерелевантности  |
| `@output_guardrail` | **После** получения ответа от агента | Проверка формата, цензуры, логики |


### 🧪 Жизненный цикл input guardrail

```text
пользователь вводит → input_guardrail → tripwire? 
                            ↓
                        [True] → прерывание + возврат info
                            ↓
                        [False] → запускается агент
```


### 🧠 Вывод

> `@input_guardrail` — это механизм **контроля входа**, позволяющий:
>
> * писать кастомную логику фильтрации,
> * использовать LLM (через Runner) внутри guardrail-а,
> * прерывать выполнение агента, если вход не соответствует критериям.

В сочетании с `GuardrailFunctionOutput`, это позволяет построить **надёжную систему безопасности** вокруг AI-агентов.

In [ ]:
# =========================
# СИСТЕМА ЗАЩИТЫ (GUARDRAILS)
# =========================
class RelevanceOutput(BaseModel):
    """
    Схема для решений guardrail о релевантности.
    
    Description:
    ---------------
        Модель данных для хранения результата проверки релевантности
        сообщения пользователя теме авиаперевозок.
    
    Args:
    ---------------
        reasoning: Обоснование решения
        is_relevant: Флаг релевантности сообщения
    """
    
    reasoning: str
    is_relevant: bool


# Агент для проверки релевантности запросов
guardrail_agent = Agent(
    model="gpt-4.1-mini",
    name="Relevance Guardrail",
    instructions=(
        "Определи, является ли сообщение пользователя совершенно не связанным "
        "с обычным разговором службы поддержки авиакомпании (рейсы, бронирования, "
        "багаж, регистрация, статус рейса, правила, программы лояльности и т.д.). "
        "Важно: Ты оцениваешь ТОЛЬКО последнее сообщение пользователя, "
        "а не предыдущие сообщения из истории чата. "
        "Допустимы сообщения типа 'Привет', 'Хорошо' или любые другие "
        "разговорные сообщения, но если ответ не разговорный, он должен "
        "хотя бы частично касаться авиаперевозок. "
        "Верни is_relevant=True если сообщение релевантно, иначе False, "
        "плюс краткое обоснование."
    ),
    output_type=RelevanceOutput,
)


# Функция-защитник для проверки релевантности
@input_guardrail(name="Relevance Guardrail")
async def relevance_guardrail(
    context: RunContextWrapper[None], 
    agent: Agent, 
    input: Union[str, List[TResponseInputItem]]
) -> GuardrailFunctionOutput:
    """
    Защитный механизм для проверки релевантности запроса.
    
    Description:
    ---------------
        Проверяет, относится ли запрос пользователя к теме
        авиаперевозок и клиентского сервиса авиакомпании.
    
    Args:
    ---------------
        context: Контекст выполнения без типизации
        agent: Агент, для которого выполняется проверка
        input: Входящее сообщение или список сообщений
    
    Returns:
    ---------------
        GuardrailFunctionOutput: Результат проверки с информацией о срабатывании
    """
    # Запускаем агент для анализа релевантности
    result = await Runner.run(guardrail_agent, input, context=context.context)
    final = result.final_output_as(RelevanceOutput)
    
    # Возвращаем результат с флагом срабатывания
    return GuardrailFunctionOutput(
        output_info=final, 
        tripwire_triggered=not final.is_relevant
    )

## 🧠 7. Что такое `handoff`

`handoff(...)` — это **механизм делегирования управления от одного агента другому**. Он превращает подагента в *tool*, доступный главному агенту (`triage_agent`), с возможностью:

* переключиться на нового агента,
* передать ему текущий контекст и историю,
* (опционально) вызвать колбэк `on_handoff`.

Это **фундаментальный строительный блок маршрутизации** в multi-agent архитектуре.


### 🧩 Архитектура `handoff` — что из чего состоит

```python
handoff(agent=seat_booking_agent, on_handoff=on_seat_booking_handoff)
```

Под капотом создаётся объект:

```python
Handoff(
    tool_name="transfer_to_seat_booking_agent",
    tool_description="Handoff to the SeatBooking agent to handle the request...",
    on_invoke_handoff=...,
    input_json_schema=...,  # пустой или с валидацией
    agent_name="SeatBooking",
    input_filter=None
)
```


### 🧪 Что происходит в рантайме

Когда главный агент (`triage_agent`) обрабатывает запрос:

1. LLM (через prompt) решает: 🧠 «мне нужно делегировать задачу агенту A».
2. Вызов `handoff_tool(...)` **выглядит как обычный tool call** для LLM:

   ```json
   {
     "tool_call": "transfer_to_seat_booking_agent",
     "arguments": "{}"
   }
   ```
3. SDK обрабатывает этот вызов:

   * вызывает `on_invoke_handoff(...)`
   * запускает нового агента (`seat_booking_agent`)
   * передаёт ему всю историю и контекст (возможно — отфильтрованную)
4. Новый агент продолжает общение с пользователем **в том же рантайме**, сохраняя целостность сессии.


### 🔄 Роль `on_handoff`

Это **функция-колбэк**, вызываемая **перед запуском нового агента**:

```python
async def on_seat_booking_handoff(context: RunContextWrapper[...]) -> None:
    context.context.flight_number = "..."
```

Она нужна, чтобы:

* модифицировать контекст;
* записать лог;
* задать начальные параметры следующему агенту.

Если задан `input_type`, то:

```python
on_handoff(ctx: RunContextWrapper, input: MyInputModel) → None
```

SDK валидирует JSON-параметры перед вызовом через `TypeAdapter`.


### 📦 Что такое `input_filter`

Агент по умолчанию получает всю историю. Но можно её отфильтровать:

```python
def my_filter(data: HandoffInputData) -> HandoffInputData:
    # например, убираем предыдущие tool-calls
    ...
```

Этот механизм позволяет построить **тонкий контроль контекста**, передаваемого следующему агенту.


### 🔐 Почему работает как `@function_tool`

В `Agent` объекте:

```python
Agent(
    handoffs=[
        handoff(agent=..., on_handoff=...)
    ]
)
```

Каждый `Handoff` регистрируется **как tool** для LLM. Это значит:

* LLM в prompt видит: «ты можешь вызвать `transfer_to_seat_booking_agent`».
* Делегирование реализуется **как вызов инструмента** (`tool_call`), но с особой реализацией.


### 📊 Пример жизненного цикла

```mermaid
sequenceDiagram
    participant User
    participant TriageAgent
    participant SDK
    participant SeatBookingAgent

    User->>TriageAgent: "Хочу выбрать место"
    TriageAgent->>SDK: tool_call → "transfer_to_seat_booking_agent"
    SDK->>on_handoff: вызов (модификация контекста)
    SDK->>SeatBookingAgent: запуск нового агента
    SeatBookingAgent->>User: "Какое место вы хотите?"
```


### ✅ Вывод

> `handoff(...)` — это не просто делегирование. Это:
>
> * **инструмент маршрутизации между агентами**, доступный как tool LLM;
> * **контейнер логики** через `on_handoff`;
> * **контроллер истории и контекста**, через `input_filter`;
> * **безопасный и строгий** механизм с optional schema validation.

Это позволяет строить сложные цепочки агентов, где каждый специализированный агент отвечает за свою зону ответственности.


In [ ]:
# Главный агент для маршрутизации запросов (triage)
triage_agent = Agent[AirlineAgentContext](
    name="Triage Agent",
    model="gpt-4.1",
    handoff_description=(
        "Агент сортировки, который может делегировать запрос клиента "
        "соответствующему агенту."
    ),
    instructions=(
        f"{RECOMMENDED_PROMPT_PREFIX} "
        "Ты полезный агент сортировки. Ты можешь использовать свои инструменты для "
        "делегирования вопросов другим подходящим агентам."
    ),
    handoffs=[
        flight_status_agent,
        handoff(agent=cancellation_agent, on_handoff=on_cancellation_handoff),
        faq_agent,
        handoff(agent=seat_booking_agent, on_handoff=on_seat_booking_handoff),
    ],
    input_guardrails=[relevance_guardrail, jailbreak_guardrail],
)

# =========================
# НАСТРОЙКА СВЯЗЕЙ МЕЖДУ АГЕНТАМИ
# =========================
# Настраиваем возможность передачи управления обратно к triage агенту
faq_agent.handoffs.append(triage_agent)
seat_booking_agent.handoffs.append(triage_agent)
flight_status_agent.handoffs.append(triage_agent)
cancellation_agent.handoffs.append(triage_agent)